In [10]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import numpy as np 

In [4]:
def data_normalize(data_temp):
    data_temp2=data_temp.T
    data_temp2 -=np.mean(data_temp2,axis=0)
    data_temp2 /=np.std(data_temp2,axis=0)
    data_temp=data_temp2.T
    return data_temp

In [ ]:
def Xload(paths):
    data_list=[]
    for path in paths:
        temp=data_normalize(np.loadtxt(path))
        data_list.append(temp)
    ######合并acc_x/y/z,gyr_x/y/z
    lenth=len(data_list[0])
    width=len(data_list[0][1])
    data_np=np.empty((lenth,width*6))
    for i in range(width*6):
        temp= i%6
        data_np[:,i]=data_list[temp][:,i//6]
    return data_np

In [ ]:
def Yload(path):
    train_np= np.loadtxt(path)  
    train_np1=list(train_np)
    train_np2=list(map(int, train_np1))
    ytrain_np=[train_value-1 for train_value in train_np2]
    #Convert tags to onehot
    batch_size = tf.size(ytrain_np)
    labels = tf.expand_dims(ytrain_np, 1)
    indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
    concated = tf.concat([indices, labels],1)
    # 2. FIT
    enc.fit(concated)
    # 3. Transform
    onehot_labels = enc.transform(concated).toarray()
    #onehot_labels = tf.sparse_to_dense(concated, tf.stack([batch_size, 21]), 1.0, 0.0)
    return onehot_labels

In [ ]:
input_types= ["acc_x.txt","acc_y.txt","acc_z.txt","gyr_x.txt","gyr_y.txt","gyr_z.txt"]
train_paths=["Dataset1/train/Inertial/"+ i for i in input_types]
train_np=Xload(train_paths)
test_paths=["Dataset1/train/Inertial/"+ i for i in input_types]
test_np=Xload(test_paths)

ytrain_path= "Dataset1/train/y_train.txt"
ytrain_label=Yload(ytrain_path)
ytest_path= "Dataset1/test/y_test.txt"
ytest_label=Yload(ytest_path)

In [ ]:
ytrain_label.shape

In [ ]:
train_np.shape

In [27]:
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [28]:
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [32]:
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial_Signals/'
	# load all 9 files as a single array
	filenames = list()
	# body acceleration
	filenames += ['acc_x.txt', 'acc_y.txt', 'acc_z.txt']
	# body gyroscope
	filenames += ['gyr_x.txt', 'gyr_y.txt', 'gyr_z.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [43]:
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'Dataset1/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'Dataset1/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [45]:
trainX, trainy, testX, testy=load_dataset()

(33104, 128, 6) (33104, 1)
(3740, 128, 6) (3740, 1)
(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [46]:
trainX.transpose(0,2,1)

array([[[-1.31394e+01, -1.11085e+01, -9.92756e+00, ..., -3.83611e+00,
         -4.75848e+00, -6.43442e+00],
        [-7.64229e+00, -5.65032e+00, -2.61088e+00, ..., -1.06243e+00,
         -6.77559e-01, -8.97826e-01],
        [-2.33387e+01, -1.66469e+01, -1.14215e+01, ..., -7.79373e+00,
         -9.43076e+00, -1.16813e+01],
        [ 3.35976e-01, -1.33169e+00, -8.14742e-01, ..., -7.96415e-02,
          1.11788e-01,  4.56851e-01],
        [-1.52838e+00, -2.49622e+00, -8.38107e-01, ...,  3.19101e-01,
          3.55524e-01,  9.23552e-01],
        [ 3.29868e-02,  3.07952e-01, -2.21133e-01, ...,  2.77943e-02,
         -1.36528e-01, -3.90877e-01]],

       [[-1.01993e+00,  3.64068e+00,  9.33020e+00, ..., -1.99003e+00,
         -1.73071e+00, -4.23475e-02],
        [-1.16454e+01, -1.36579e+01, -1.57544e+01, ..., -1.67745e+01,
         -1.43318e+01, -1.28019e+01],
        [ 2.30322e+00,  2.77480e+00,  4.43077e+00, ...,  5.07945e+00,
          4.02450e+00,  2.69797e+00],
        [-9.77385e-03,  9.

In [47]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('trainX.npy', trainX)
#np.load(tranx.npy)

In [48]:
# load numpy array from npy file
from numpy import load
# load array
trainX = load('trainX.npy')
# print the array
print(trainX)

[[[-1.31394e+01 -7.64229e+00 -2.33387e+01  3.35976e-01 -1.52838e+00
    3.29868e-02]
  [-1.11085e+01 -5.65032e+00 -1.66469e+01 -1.33169e+00 -2.49622e+00
    3.07952e-01]
  [-9.92756e+00 -2.61088e+00 -1.14215e+01 -8.14742e-01 -8.38107e-01
   -2.21133e-01]
  ...
  [-3.83611e+00 -1.06243e+00 -7.79373e+00 -7.96415e-02  3.19101e-01
    2.77943e-02]
  [-4.75848e+00 -6.77559e-01 -9.43076e+00  1.11788e-01  3.55524e-01
   -1.36528e-01]
  [-6.43442e+00 -8.97826e-01 -1.16813e+01  4.56851e-01  9.23552e-01
   -3.90877e-01]]

 [[-1.01993e+00 -1.16454e+01  2.30322e+00 -9.77385e-03 -1.52716e-01
   -4.66701e-01]
  [ 3.64068e+00 -1.36579e+01  2.77480e+00  9.67553e-02  1.73591e-01
   -4.24294e-01]
  [ 9.33020e+00 -1.57544e+01  4.43077e+00  2.15826e-01  1.85169e-01
   -1.21009e-01]
  ...
  [-1.99003e+00 -1.67745e+01  5.07945e+00  6.87930e-01 -3.98198e-01
    8.04290e-01]
  [-1.73071e+00 -1.43318e+01  4.02450e+00  4.31328e-01 -2.17869e-01
    5.17097e-01]
  [-4.23475e-02 -1.28019e+01  2.69797e+00 -1.97109e

In [49]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('trainX.npy', trainX)
#np.load(tranx.npy)